```
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   ██████╗ ██████╗ ███████╗    ████████╗ ██████╗     █████╗ ██╗                ║
║   ██╔══██╗██╔══██╗██╔════╝    ╚══██╔══╝██╔═══██╗   ██╔══██╗██║                ║
║   ██████╔╝██║  ██║█████╗         ██║   ██║   ██║   ███████║██║                ║
║   ██║     ██║  ██║██╔══╝         ██║   ██║   ██║   ██╔══██║██║                ║
║   ██║     ██████╔╝██║            ██║   ╚██████╔╝   ██║  ██║██║                ║
║   ╚═╝     ╚═════╝ ╚═╝            ╚═╝    ╚═════╝    ╚═╝  ╚═╝╚═╝                ║
║                                                                               ║
║          From Structured Output to "Just Tell the AI"                         ║
║                                                                               ║
║                    Unattended 2025 - Chrissy LeMaire                          ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
```

---
## The Evolution

> "I used to spend hours teaching you how to parse PDFs into structured output..."

**Then:** JSON schemas, strict validation, prompt engineering, field mapping, error handling

**Now:** `"Review this PDF, create normalized tables, and import the data"`

Let's see Claude do the heavy lifting.

In [1]:
# Anthropic's having outages. PIVOTTT
Set-AIToolDefault -Tool Copilot

# Setup
$PSDefaultParameterValues["Invoke-AITool:Raw"] = $true
$PSDefaultParameterValues["Invoke-AITool:Model"] = "claude-sonnet-4.5"

$sqlInstance = "localhost"
$database = "tempdb"

key,value
FullName,AITools.DefaultTool
Value,Copilot


In [2]:
# Let's see what we're working with
Start-Process ./Tests/pdf/immunization.pdf

---
## Part 1: Schema Design

No pre-planning. No schema files. Just ask Claude to analyze the PDF and design proper normalized tables.

> "Show me your work, then create it."

In [ ]:
$designPrompt = @"
Review Tests/pdf/immunization.pdf carefully and design a normalized database schema.

Requirements:
- Create properly normalized tables (at least 2 tables with foreign key relationship)
- Use best practices for data types, constraints, and indexing
- Target DB: SQL Server, localhost, windows auth, tempdb

Output:
1. First, explain your design decisions (table structure, why you normalized this way)
2. Then provide the complete T-SQL DDL to create the tables
3. Execute ONLY DDL against local SQL Server
"@

Invoke-AITool -Prompt $designPrompt

---
## Part 2: Extract (The Annoying ETL Part)

Now the classic ETL challenge: extract data from the PDF and insert into normalized tables.

First pass: **Extract exactly as-is.** Bad data and all.

In [ ]:
$extractPrompt = "Extract the data from /Tests/pdf/immunization.md

I have vaccine tables in tempdb on localhost (sql server/windows auth),
figure out the structure, insert the data from the PDF then show me what
you inserted"

Invoke-AITool -Prompt $extractPrompt

### Go look in SSMS, notice anything wrong?

The data is in there exactly as the PDF showed it. But there might be problems...

---
## Part 3: Upsert with AI Data Quality Review

Now the magic. Ask Claude to review the data, identify problems, fix them, AND document what it found.

In [ ]:
Invoke-AITool -Prompt C:\github\aitools\unattended.md -Tool Claude

---
## Part 4: Data Quality Report

Generate a markdown report documenting all the issues found. 

This is your audit trail - proof that AI caught problems human review might miss.

In [ ]:
$reportPrompt = @"
Review /Tests/pdf/immunization.pdf carefully and generate a data quality report.

Analyze the document thoroughly for:
- Missing or incomplete information
- Suspicious or potentially incorrect values
- Format issues
- Any other data quality concerns

Output a well-formatted Markdown report with:
- Executive summary
- Table of issues found (columns: Field, Value in PDF, Issue, Recommended Fix)
- Confidence level for each finding
- Recommendations for human review

Be thorough - this report helps humans catch shit data before it pollutes the database.
"@

$report = Invoke-AITool -Prompt $reportPrompt

# Save the report
$report | Set-Content ./Tests/pdf/data_quality_report.md
Write-Host "Report saved to: ./Tests/pdf/data_quality_report.md" -ForegroundColor Green

# Display it
$report

---
## The Point

### What I Used to Teach
```powershell
# 1. Define rigid JSON schema (50 lines)
# 2. Configure structured output API
# 3. Write validation rules (100+ lines)
# 4. Build table mappings
# 5. Handle edge cases
# 6. Debug parsing failures
# 7. Hope you thought of everything
```

### What I Do Now
```powershell
"Review this PDF, create normalized tables, import the data, 
 find the problems, fix them, and document what you found."
```

---

### Why This Works

| Traditional ETL | AI-Assisted ETL |
|-----------------|----------------|
| Check against rules I thought of | Notices things I didn't think to check |
| Rigid schema matching | Semantic understanding |
| Fails on edge cases | Adapts to messy data |
| Silent data corruption | Documents concerns for review |

**The key insight:** AI catches semantic issues that rule-based validation can't.

"Labrador" is a valid breed. Traditional validation would pass it.

But AI looks at the *whole picture* and knows it doesn't quite add up.